### Numerical Computation of Multidimensional Integrals Using Gauss-Hermite Quadrature Rules

Here I follow Mark van der Wilk's [notes](https://gist.github.com/markvdw/f9ca12c99484cf2a881e84cb515b86c8) and extend to the case of a lognormal distribution.

TODO: Add introduction and change of variables.

TODO: Add reference to Chris Conlon's lecture notes and videos.

In [1]:
# Import libraries
import numpy as np
from scipy.special import roots_hermite
from scipy.stats import lognorm

In [2]:
# Example from Mark van der Wilk's notes
μ = np.array([1, 0])
Σ = np.array([[1.3, -0.213], [-0.213, 1.2]])
L = np.linalg.cholesky(Σ)
N = 2
c = np.pi**(-0.5*N)
x, w = roots_hermite(100)

X1, X2 = np.meshgrid(x, x)
W1, W2 = np.meshgrid(w, w)

X = np.vstack((X1.flatten(), X2.flatten())).T
W = np.multiply(W1.flatten(), W2.flatten())


Y = np.sqrt(2)*(L@X.T).T + μ[None, :]

# Check that weights add up to one
np.sum(c*W)

# Check that the mean is correct
μ_res = c*W@Y
print(f'Estimated μ: {μ_res}')

# Check that the covariance is correct
cov = lambda x: np.outer(x-μ, x-μ)
F = np.array([cov(y) for y in Y])
Σ_res = np.sum(c*W[:, None, None]*F, axis=0)
print(f'Estimated Σ: {Σ_res}')


Estimated μ: [1.00000000e+00 2.77555756e-16]
Estimated Σ: [[ 1.3   -0.213]
 [-0.213  1.2  ]]


## Let's now try with bivariate lognormal

Suppose $x, y$ are distributed joint lognormal so that $\log{x}, \log{y}$ are distributed jointly normal with the parametrization above.

The expected values, variances and covariances are given by

- $\mathbb{E}[x] = \exp({\mu_x+\frac{1}{2}V_x})$
- $\mathbb{E}[y] = \exp({\mu_y+\frac{1}{2}V_y})$ 
- $\mathbb{V}[x]=\mathbb{E}[x]^2(\exp(V_x)-1)$
- $\mathbb{V}[y]=\mathbb{E}[y]^2(\exp(V_y)-1)$
- $\mathbb{Cov}[x, y]=\mathbb{E}[x]\mathbb{E}[y](\exp(C)-1)$

where $\mu_x=1$, $\mu_y=0$, $V_x=1.3$, $V_y=1.2$, $C=-0.213$ are the parameters of the joint normal distribution in the first example.


In [3]:
# Obtain the true values of means and variances from Python (doesn't give covariance)
mean_py_x = lognorm.mean(s=np.sqrt(Σ[0, 0]), scale=np.exp(μ[0]))
mean_py_y = lognorm.mean(s=np.sqrt(Σ[1, 1]), scale=np.exp(μ[1]))
mean_py = np.array([mean_py_x, mean_py_y])

var_true_x = lognorm.var(s=np.sqrt(Σ[0, 0]), scale=np.exp(μ[0]))
var_true_y = lognorm.var(s=np.sqrt(Σ[1, 1]), scale=np.exp(μ[1]))

print(f'True means py: {mean_py}\n')

print(f'True variances py: {var_true_x}, {var_true_y}\n')

# Obtain analytic true values (see notes on Veiga and Weyl(2016))
mean_x = np.exp(μ[0]+.5*Σ[0, 0])
mean_y = np.exp(μ[1]+.5*Σ[1, 1])
var_x = mean_x**2*(np.exp(Σ[0, 0])-1)
var_y = mean_y**2*(np.exp(Σ[1, 1])-1)
cov_x_y = mean_x*mean_y*(np.exp(Σ[0, 1])-1)
mean = np.array([mean_x, mean_y]) 

print(f'True means analytic: {mean_x}, {mean_y}\n')

print(f'True variances analytic: {var_x}, {var_y}\n')

print(f'True covariance analytic: {cov_x_y}\n')


# Compute estimations using quadrature
mean_res = c*W@np.exp(Y)
print(f'Estimated μ: {mean_res}\n')

cov_ln = lambda x: np.outer(np.exp(x)-mean, np.exp(x)-mean)
F_ln = np.array([cov_ln(y) for y in Y])
Var_res = np.sum(c*W[:, None, None]*F_ln, axis=0)
print(f'Estimated Σ: {Var_res}\n')

True means py: [5.20697983 1.8221188 ]

True variances py: 72.37167672127596, 7.70305945790505

True means analytic: 5.206979827179849, 1.8221188003905089

True variances analytic: 72.37167672127592, 7.703059457905053

True covariance analytic: -1.8201638919165706

Estimated μ: [5.20697983 1.8221188 ]

Estimated Σ: [[72.37167672 -1.82016389]
 [-1.82016389  7.70305946]]

